In [1]:
#In this cell we import all the libraries we need for this project

import numpy as np

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim

import requests 
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0
import folium 

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    certifi-2019.6.16          |           py36_1         149 KB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [2]:
#In this cell we load the New York City neighborhood data

!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [3]:
#In this cell we extract just the necessary neighborhood data from the New York City dataset

with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
neighborhoods_data = newyork_data['features']

In [4]:
#In this cell we create a DataFrame with only the borough information

column_names = ['Borough'] 
boroughs = pd.DataFrame(columns=column_names)

In [5]:
#In this cell we populate that DataFrame and return an array of the five NYC boroughs

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough']
    boroughs = boroughs.append({'Borough': borough}, ignore_index=True)
boroughs.Borough.unique()

array(['Bronx', 'Manhattan', 'Brooklyn', 'Queens', 'Staten Island'],
      dtype=object)

In [6]:
#In this cell we define FourSquare credentials

CLIENT_ID = 'OD52D2OKTNM0M0H4NJGTXCKQSJXK02ZMY1YRCHJ23NF3V0GT'
CLIENT_SECRET = 'DOC2PICIMX4IPFVYXICBYQGHYADDIPOI2PA1UY45422O3TUZ'
VERSION = '20180605'

In [7]:
#In this cell we search for the term "Hospital" around a 1 mile radius of our first borough, The Bronx
address = 'Bronx, NY'
latitude = 40.8448
longitude = -73.8648
search_query='Hospital'
LIMIT = 100 
radius = 1610

url1 = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude,
    search_query,
    radius, 
    LIMIT)
url1 

'https://api.foursquare.com/v2/venues/search?&client_id=OD52D2OKTNM0M0H4NJGTXCKQSJXK02ZMY1YRCHJ23NF3V0GT&client_secret=DOC2PICIMX4IPFVYXICBYQGHYADDIPOI2PA1UY45422O3TUZ&v=20180605&ll=40.8448,-73.8648&query=Hospital&radius=1610&limit=100'

In [8]:
#In this cell we can see the results for hospitals near The Bronx

results = requests.get(url1).json()
results

{'meta': {'code': 200, 'requestId': '5d680924bbed2100327e397f'},
 'response': {'venues': [{'id': '4ca63f8ef47ea143bec25e21',
    'name': 'Animal Hospital of Morris Park',
    'location': {'address': '1135 Morris Park Avenue',
     'crossStreet': 'Williamsbridge Rd',
     'lat': 40.85024471419821,
     'lng': -73.85180468158171,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.85024471419821,
       'lng': -73.85180468158171}],
     'distance': 1250,
     'postalCode': '10461',
     'cc': 'US',
     'city': 'Bronx',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['1135 Morris Park Avenue (Williamsbridge Rd)',
      'Bronx, NY 10461',
      'United States']},
    'categories': [{'id': '4d954af4a243a5684765b473',
      'name': 'Veterinarian',
      'pluralName': 'Veterinarians',
      'shortName': 'Veterinarians',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/medical_veterinarian_',
       'suffix': '.png'},
      'pr

In [9]:
#In this cell we define a way for us to view the category of each venue

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        print ('error')
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [10]:
#In this cell we filter and clean the columns so we can see our results clearly

venues = results['response']['venues']
nearby_hospitals1 = json_normalize(venues)

filtered_columns = ['borough','name', 'categories', 'location.lat', 'location.lng']
nearby_hospitals1 =nearby_hospitals1.loc[:, filtered_columns]
nearby_hospitals1['borough'] = address
nearby_hospitals1['categories'] = nearby_hospitals1.apply(get_category_type, axis=1)
nearby_hospitals1.columns = [col.split(".")[-1] for col in nearby_hospitals1.columns]

nearby_hospitals1

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/indexing.py:1494: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,borough,name,categories,lat,lng
0,"Bronx, NY",Animal Hospital of Morris Park,Veterinarian,40.850245,-73.851805
1,"Bronx, NY",Calvary Hospital,Hospital,40.848169,-73.843860
2,"Bronx, NY",Bronx County Animal Hospital,Veterinarian,40.847311,-73.857398
3,"Bronx, NY",Einstein Hospital Radiology Dept,Hospital,40.848729,-73.846820
4,"Bronx, NY",Jack D. Weiler Hospital ER,Emergency Room,40.846786,-73.846786
5,"Bronx, NY",Montefiore Medical Center,Hospital,40.849072,-73.846181
6,"Bronx, NY",Avilon Animal Hospital,Veterinarian,40.831988,-73.865793
7,"Bronx, NY",Avalon Animal Hospital,Veterinarian,40.831811,-73.865587
8,"Bronx, NY",Peds ICU @ Jacobi Hospital,Hospital Ward,40.839895,-73.882080
9,"Bronx, NY",Westchester Square Medical Center,Hospital,40.840897,-73.848290


In [11]:
#In this cell we repeat the search for our second borough, Manhattan

address = 'Manhattan, NY'
latitude = 40.7831
longitude = -73.9712
search_query='Hospital'
LIMIT = 100 
radius = 1610

url2 = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude,
    search_query,
    radius, 
    LIMIT)
results = requests.get(url2).json()
venues = results['response']['venues']
nearby_hospitals2 = json_normalize(venues)

filtered_columns = ['borough','name', 'categories', 'location.lat', 'location.lng']
nearby_hospitals2 =nearby_hospitals2.loc[:, filtered_columns]
nearby_hospitals2['borough'] = address
nearby_hospitals2['categories'] = nearby_hospitals2.apply(get_category_type, axis=1)
nearby_hospitals2.columns = [col.split(".")[-1] for col in nearby_hospitals2.columns]
nearby_hospitals2

,borough,name,categories,lat,lng
0,"Manhattan, NY",The Mount Sinai Hospital,Hospital,40.789811,-73.952880
1,"Manhattan, NY",Lenox Hill Hospital,Hospital,40.773796,-73.961115
2,"Manhattan, NY",Kravis Children's Hospital,Hospital,40.790095,-73.953917
3,"Manhattan, NY",Lenox Hill Hospital Cafeteria,Café,40.773333,-73.961024
4,"Manhattan, NY",Victory Gardens Lenox Hill Hospital,Scenic Lookout,40.773241,-73.961024
5,"Manhattan, NY",CCU @ Lenox Hill Hospital,Medical Center,40.773360,-73.960956
6,"Manhattan, NY","Mount Sinai Hospital, Nephrology",Hospital,40.786903,-73.953880
7,"Manhattan, NY",Mount Sinai Hospital Department of Genetics an...,Hospital,40.789406,-73.953173
8,"Manhattan, NY",St. Luke's Hospital - Rosevelt Division,Hospital,40.788202,-73.972486
9,"Manhattan, NY",Hudson Animal Hospital,Veterinarian,40.772405,-73.988686


In [62]:
#We then append these results to our previous results
nearby_hospitals = [nearby_hospitals1, nearby_hospitals2]
nearby_hospitals= pd.concat(nearby_hospitals, ignore_index=True)
nearby_hospitals

,borough,name,categories,lat,lng
0,"Bronx, NY",Animal Hospital of Morris Park,Veterinarian,40.850245,-73.851805
1,"Bronx, NY",Calvary Hospital,Hospital,40.848169,-73.843860
2,"Bronx, NY",Bronx County Animal Hospital,Veterinarian,40.847311,-73.857398
3,"Bronx, NY",Einstein Hospital Radiology Dept,Hospital,40.848729,-73.846820
4,"Bronx, NY",Jack D. Weiler Hospital ER,Emergency Room,40.846786,-73.846786
5,"Bronx, NY",Montefiore Medical Center,Hospital,40.849072,-73.846181
6,"Bronx, NY",Avilon Animal Hospital,Veterinarian,40.831988,-73.865793
7,"Bronx, NY",Avalon Animal Hospital,Veterinarian,40.831811,-73.865587
8,"Bronx, NY",Peds ICU @ Jacobi Hospital,Hospital Ward,40.839895,-73.882080
9,"Bronx, NY",Westchester Square Medical Center,Hospital,40.840897,-73.848290


In [12]:
#We then repeat this process for the third borough, Brooklyn

address = 'Brooklyn, NY'
latitude = 40.6782
longitude = -73.9442
search_query='Hospital'
LIMIT = 100 
radius = 1610

url3 = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude,
    search_query,
    radius, 
    LIMIT)
results = requests.get(url3).json()
venues = results['response']['venues']
nearby_hospitals3 = json_normalize(venues)

filtered_columns = ['borough','name', 'categories', 'location.lat', 'location.lng']
nearby_hospitals3 =nearby_hospitals3.loc[:, filtered_columns]
nearby_hospitals3['borough'] = address
nearby_hospitals3['categories'] = nearby_hospitals3.apply(get_category_type, axis=1)
nearby_hospitals3.columns = [col.split(".")[-1] for col in nearby_hospitals3.columns]

nearby_hospitals = [nearby_hospitals1, nearby_hospitals2, nearby_hospitals3]
nearby_hospitals= pd.concat(nearby_hospitals, ignore_index=True)
nearby_hospitals

,borough,name,categories,lat,lng
0,"Bronx, NY",Animal Hospital of Morris Park,Veterinarian,40.850245,-73.851805
1,"Bronx, NY",Calvary Hospital,Hospital,40.848169,-73.843860
2,"Bronx, NY",Bronx County Animal Hospital,Veterinarian,40.847311,-73.857398
3,"Bronx, NY",Einstein Hospital Radiology Dept,Hospital,40.848729,-73.846820
4,"Bronx, NY",Jack D. Weiler Hospital ER,Emergency Room,40.846786,-73.846786
5,"Bronx, NY",Montefiore Medical Center,Hospital,40.849072,-73.846181
6,"Bronx, NY",Avilon Animal Hospital,Veterinarian,40.831988,-73.865793
7,"Bronx, NY",Avalon Animal Hospital,Veterinarian,40.831811,-73.865587
8,"Bronx, NY",Peds ICU @ Jacobi Hospital,Hospital Ward,40.839895,-73.882080
9,"Bronx, NY",Westchester Square Medical Center,Hospital,40.840897,-73.848290


In [13]:
#And for the fourth borough of Queens

address = 'Queens, NY'
latitude = 40.7282
longitude = -73.7949
search_query='Hospital'
LIMIT = 100 
radius = 1610

url4 = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude,
    search_query,
    radius, 
    LIMIT)
results = requests.get(url4).json()
venues = results['response']['venues']
nearby_hospitals4 = json_normalize(venues)

filtered_columns = ['borough','name', 'categories', 'location.lat', 'location.lng']
nearby_hospitals4 =nearby_hospitals4.loc[:, filtered_columns]
nearby_hospitals4['borough'] = address
nearby_hospitals4['categories'] = nearby_hospitals4.apply(get_category_type, axis=1)
nearby_hospitals4.columns = [col.split(".")[-1] for col in nearby_hospitals4.columns]

nearby_hospitals = [nearby_hospitals1, nearby_hospitals2, nearby_hospitals3, nearby_hospitals4]
nearby_hospitals= pd.concat(nearby_hospitals, ignore_index=True)
nearby_hospitals

,borough,name,categories,lat,lng
0,"Bronx, NY",Animal Hospital of Morris Park,Veterinarian,40.850245,-73.851805
1,"Bronx, NY",Calvary Hospital,Hospital,40.848169,-73.843860
2,"Bronx, NY",Bronx County Animal Hospital,Veterinarian,40.847311,-73.857398
3,"Bronx, NY",Einstein Hospital Radiology Dept,Hospital,40.848729,-73.846820
4,"Bronx, NY",Jack D. Weiler Hospital ER,Emergency Room,40.846786,-73.846786
5,"Bronx, NY",Montefiore Medical Center,Hospital,40.849072,-73.846181
6,"Bronx, NY",Avilon Animal Hospital,Veterinarian,40.831988,-73.865793
7,"Bronx, NY",Avalon Animal Hospital,Veterinarian,40.831811,-73.865587
8,"Bronx, NY",Peds ICU @ Jacobi Hospital,Hospital Ward,40.839895,-73.882080
9,"Bronx, NY",Westchester Square Medical Center,Hospital,40.840897,-73.848290


In [14]:
#And then for Staten Island

address = 'Staten Island, NY'
latitude = 40.5795
longitude = -74.1502
print('The geograpical coordinates of Staten Island are {}, {}.'.format(latitude, longitude))
search_query='Hospital'
LIMIT = 100 
radius = 1610

url5 = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude,
    search_query,
    radius, 
    LIMIT)
results = requests.get(url5).json()
venues = results['response']['venues']
nearby_hospitals5 = json_normalize(venues)

filtered_columns = ['borough','name', 'categories', 'location.lat', 'location.lng']
nearby_hospitals5 =nearby_hospitals5.loc[:, filtered_columns]
nearby_hospitals5['borough'] = address
nearby_hospitals5['categories'] = nearby_hospitals5.apply(get_category_type, axis=1)
nearby_hospitals5.columns = [col.split(".")[-1] for col in nearby_hospitals5.columns]

nearby_hospitals = [nearby_hospitals1, nearby_hospitals2, nearby_hospitals3, nearby_hospitals4, nearby_hospitals5]
nearby_hospitals= pd.concat(nearby_hospitals, ignore_index=True)
nearby_hospitals

The geograpical coordinates of Staten Island are 40.5795, -74.1502.


,borough,name,categories,lat,lng
0,"Bronx, NY",Animal Hospital of Morris Park,Veterinarian,40.850245,-73.851805
1,"Bronx, NY",Calvary Hospital,Hospital,40.848169,-73.843860
2,"Bronx, NY",Bronx County Animal Hospital,Veterinarian,40.847311,-73.857398
3,"Bronx, NY",Einstein Hospital Radiology Dept,Hospital,40.848729,-73.846820
4,"Bronx, NY",Jack D. Weiler Hospital ER,Emergency Room,40.846786,-73.846786
5,"Bronx, NY",Montefiore Medical Center,Hospital,40.849072,-73.846181
6,"Bronx, NY",Avilon Animal Hospital,Veterinarian,40.831988,-73.865793
7,"Bronx, NY",Avalon Animal Hospital,Veterinarian,40.831811,-73.865587
8,"Bronx, NY",Peds ICU @ Jacobi Hospital,Hospital Ward,40.839895,-73.882080
9,"Bronx, NY",Westchester Square Medical Center,Hospital,40.840897,-73.848290


In [15]:
#In this cell we remove anything that does not have the category "Hospital", "Medical Center", or "Emergency Room"

not_hospitals = nearby_hospitals[ (nearby_hospitals['categories'] != 'Hospital') & (nearby_hospitals['categories'] != 'Medical Center') & (nearby_hospitals['categories'] != 'Emergency Room')].index
nearby_hospitals.drop(not_hospitals , inplace=True)
nearby_hospitals

,borough,name,categories,lat,lng
1,"Bronx, NY",Calvary Hospital,Hospital,40.848169,-73.843860
3,"Bronx, NY",Einstein Hospital Radiology Dept,Hospital,40.848729,-73.846820
4,"Bronx, NY",Jack D. Weiler Hospital ER,Emergency Room,40.846786,-73.846786
5,"Bronx, NY",Montefiore Medical Center,Hospital,40.849072,-73.846181
9,"Bronx, NY",Westchester Square Medical Center,Hospital,40.840897,-73.848290
10,"Manhattan, NY",The Mount Sinai Hospital,Hospital,40.789811,-73.952880
11,"Manhattan, NY",Lenox Hill Hospital,Hospital,40.773796,-73.961115
12,"Manhattan, NY",Kravis Children's Hospital,Hospital,40.790095,-73.953917
15,"Manhattan, NY",CCU @ Lenox Hill Hospital,Medical Center,40.773360,-73.960956
16,"Manhattan, NY","Mount Sinai Hospital, Nephrology",Hospital,40.786903,-73.953880


In [16]:
#In this cell we can see the total number of hospitals that our search returned

print(nearby_hospitals.shape)

(47, 5)


In [17]:
#In this cell we can see the total number of hospitals that our search returned for each borough

newyork_numbers=nearby_hospitals.groupby('borough').count()
newyork_numbers.head()

,name,categories,lat,lng
borough,,,,
"Bronx, NY",5,5,5,5
"Brooklyn, NY",10,10,10,10
"Manhattan, NY",20,20,20,20
"Queens, NY",11,11,11,11
"Staten Island, NY",1,1,1,1


In [18]:
#In this cell we map the hospitals so we can visualize the density within each borough

map_newyork_hospitals = folium.Map(location=[40.7128, -74.0060], zoom_start=11)

for borough, name, cat, lat, lng in zip(nearby_hospitals['borough'], nearby_hospitals['name'], nearby_hospitals['categories'], nearby_hospitals['lat'], nearby_hospitals['lng']):
    label = '{},{}'.format(name, cat)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork_hospitals)  
map_newyork_hospitals